#  RAG의 기본 개념 / 문서 전처리 과정의 이해

### **학습 목표:**  RAG 아키텍처를 이해하고  전체 파이프라인을 설계할 수 있다

---

# 환경 설정 및 준비

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# RAG (Retrieval-Augmented Generation)

1. **RAG란**?

    - RAG는 기존 LLM의 한계를 보완하기 위한 방법론
    - 외부 데이터를 검색하여 LLM의 지식을 보강하는 방식으로 동작 (기존의 언어 모델에 검색 기능을 추가)
    - 최신 정보나 특정 도메인의 전문 지식을 활용 가능
    - 주어진 질문이나 문제에 대해 더 정확하고 풍부한 정보를 기반으로 답변을 생성

2. **RAG의 장점**
    - 환각(Hallucination) 감소
    - 최신 정보 반영 가능
    - 도메인 특화된 응답 생성
    - 소스 추적 가능성 확보

3. **핵심 구성요소**

    1. 검색(Retrieval) 시스템
        - 임베딩 모델: 텍스트를 벡터로 변환
        - 벡터 데이터베이스: 임베딩 벡터를 저장 (인덱싱)
        - 유사도 검색 알고리즘:
            * Cosine Similarity
            * Euclidean Distance
            * Dot Product

    2. 증강(Augmentation)
        - 검색된 문서의 전처리 
        - 프롬프트 엔지니어링 기법
        - 컨텍스트 길이 제한 관리

    3. 생성(Generation) 단계
        - LLM의 기본 동작으로 응답 생성 
        - 응답 형식 제어 

___

### Step 1: **Indexing**

1. 문서 수집 및 전처리
2. 문서 청크 분할
3. 임베딩 생성
4. 벡터 저장소 구축

![RAG Indexing](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png "RAG - Indexing")

`1. 문서 데이터 로드(Load Data)`

- RAG에 사용할 데이터를 불러오는 단계 (검색에 사용될 지식이나 정보)
- 외부 데이터 소스에서 정보를 수집하고, 필요한 형식으로 변환하여 시스템에 로드

- 설치: pip install langchain_community beautifulsoup4 또는 uv add langchain_community beautifulsoup4

In [ ]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader  # type: ignore

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Document 객체로 변환
docs = loader.load() 

# 결과 확인
print(f"Document 개수: {len(docs)}")
print(f"Document 길이: {len(docs[0].page_content)}")
print(f"Document 내용: {docs[0].page_content[5000:6000]}")

In [ ]:
docs[0] # Document 객체 확인

In [ ]:
docs[0].metadata # Document의 메타데이터 확인

In [ ]:
docs[0].page_content # Document의 페이지 내용 확인

`2. 문서 청크 분할(Split Texts)`

- 불러온 데이터를 작은 크기의 단위(chunk)로 분할하는 과정
- 자연어 처리(NLP) 기술을 활용하여 큰 문서를 처리가 쉽도록 문단, 문장 또는 구 단위로 나누는 작업
- 검색 효율성을 높이기 위한 중요한 과정

    1. 청크 크기 선택
        - 너무 작은 청크: 문맥 손실
        - 너무 큰 청크: 관련성 저하

    2. 중복 영역 설정
        - 문맥 유지를 위해 필요
        - 일반적으로 10-20% 권장

- 설치: pip install langchain_text_splitters 또는 uv add langchain_text_splitters

In [ ]:
# Text Split (Documents -> small chunks: Documents)
from langchain_text_splitters import CharacterTextSplitter  # type: ignore

# 1000자씩 잘라서 200자씩 겹치는 Document로 변환
text_splitter = CharacterTextSplitter(
    separator="\n\n",    # 문단 구분자
    chunk_size=1000,     # 문단 길이
    chunk_overlap=200,   # 겹치는 길이
    length_function=len, # 길이 측정 함수
    is_separator_regex=False,   # separator가 정규식인지 여부
)

splitted_docs = text_splitter.split_documents(docs)

# 결과 확인
print(f"Document 개수: {len(splitted_docs)}")
print("\n\n")

for i, doc in enumerate(splitted_docs):
    print(f"Document {i} 길이: {len(doc.page_content)}")
    print(f"Document {i} 내용: {doc.page_content[:100]}...")
    print("-"*50)

In [ ]:
# 글자 수 기준으로 엄격하게 분할하기 
from langchain_text_splitters import CharacterTextSplitter  # type: ignore

# 1000자씩 잘라서 Document로 변환
text_splitter = CharacterTextSplitter(
    separator="",        # 문단 구분자
    chunk_size=1000,     # 문단 길이
    length_function=len, # 길이 측정 함수
    is_separator_regex=False,   # separator가 정규식인지 여부
)

equally_splitted_docs = text_splitter.split_documents(docs)

# 결과 확인
print(f"Document 개수: {len(equally_splitted_docs)}")
print("\n\n")

for i, doc in enumerate(equally_splitted_docs):
    print(f"Document {i} 길이: {len(doc.page_content)}")
    print("-"*50)

`3. 문서 임베딩 생성(Document Embeddings)`

- 임베딩 모델을 사용하여 텍스트를 벡터로 변환
- 임베딩을 기반으로 유사성 검색에 사용
- 임베딩 모델 선택
   - 성능과 비용 고려
   - 다국어 지원 여부 확인

In [ ]:
# OpenAI Embeddings - 문장 임베딩

from langchain_openai import OpenAIEmbeddings  # type: ignore

# embedding model 생성
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름을 지정 가능 
)

sample_text = "위키피디아 정책 변경 절차를 알려주세요"
embedding_vector = embedding_model.embed_query(sample_text)
print(f"임베딩 벡터의 차원: {len(embedding_vector)}")
print(f"임베딩 벡터: {embedding_vector[:10]}...")

`4. 벡터 저장소 구축 (Vectorstores)`

- 임베딩 벡터를 벡터저장소에 저장 
- 저장된 임베딩을 기반으로 유사성 검색을 수행하는데 활용 

In [ ]:
# In-memory 벡터 저장소에 문서 저장하기
from langchain_core.vectorstores import InMemoryVectorStore # type: ignore
vector_store = InMemoryVectorStore(embedding_model)

# Document를 VectorStore에 저장
document_ids = vector_store.add_documents(splitted_docs)

# 결과 확인
print(f"저장된 Document 개수: {len(document_ids)}")

In [ ]:
print(f"저장된 Document ID: {document_ids[:5]}...")

In [ ]:
vector_store.store.keys() # 저장된 Document ids 확인

In [ ]:
# VectorStore에 저장된 Document 개수 확인
print(f"VectorStore에 저장된 Document 개수: {len(vector_store.store.keys())}")

In [ ]:
# 문서 삭제하기
vector_store.delete(ids=list(vector_store.store.keys())[:1])

# VectorStore에 저장된 Document 개수 확인
print(f"VectorStore에 저장된 Document 개수: {len(vector_store.store.keys())}")

### Step 2: **Retrieval and generation**

5. 검색 및 생성

![RAG Retrieval and generation](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png "RAG - Retrieval and generation")

In [ ]:
# 벡터 스토어 문서 검색 - 유사도 기반 검색

search_query = "위키피디아 정책 변경 절차를 알려주세요"

results = vector_store.similarity_search(query=search_query, k=2)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

In [ ]:
# 벡터 스토어 검색기 설정 - 유사도 기반 검색

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

# 검색기로 검색하기
results = retriever.invoke(input=search_query)

# 결과 확인
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

In [ ]:
# QA 체인 구성
from langchain.chains import RetrievalQA   # type: ignore
from langchain_openai import ChatOpenAI    # type: ignore

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4.1-mini"),
    chain_type="stuff",
    retriever=retriever,
)

# 질문 응답
query = "위키피디아 정책 변경 절차를 알려주세요"
response = qa_chain.invoke(query)

# 결과 확인
print(f"Q: {response['query']}")
print(f"A: {response['result']}")

# [실습 프로젝트]

### RAG 파이프라인 구축하기

1. 데이터 준비 → 텍스트 처리 → 임베딩 → 검색 → 평가
2. 각 과제마다 구현해야 할 함수와 결과물 정의
3. 단위 테스트를 통한 검증

In [ ]:
# 1단계: 데이터 준비 - 웹문서 검색을 위해 관련 URL 가져오기
web_urls = [
    "https://n.news.naver.com/mnews/article/029/0002927209",
    "https://n.news.naver.com/mnews/article/092/0002358620",
    "https://n.news.naver.com/mnews/article/008/0005136824",
]

In [ ]:
# 2단계: WebBaseLoader를 사용해 텍스트 로드
pass

In [ ]:
# 3단계: CharacterTextSplitter로 문서 분할
pass

In [ ]:
# 4단계: 임베딩 및 벡터 저장소 구현
pass

In [ ]:
# 5단계: RAG 기반 QA 체인 구현
pass

In [ ]:
# 6단계: QA 체인으로 질문 응답
pass